In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('Movie').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/29 22:53:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/12/29 22:53:03 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/12/29 22:53:03 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/12/29 22:53:03 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [3]:
data = spark.read.csv('movielens_ratings.csv', inferSchema=True, header=True)

In [4]:
data.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



In [5]:
from pyspark.ml.recommendation import ALS

In [6]:
from pyspark.ml.evaluation import RegressionEvaluator

In [8]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [9]:
training, test = data.randomSplit([0.8,0.2])

In [10]:
als = ALS(maxIter=5 ,regParam=0.01, userCol = 'userId',itemCol='movieId', ratingCol='rating')

In [11]:
model = als.fit(training)

23/12/29 23:02:47 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/12/29 23:02:47 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS
23/12/29 23:02:47 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [12]:
prediction = model.transform(test)

In [13]:
prediction.show()

+-------+------+------+-----------+
|movieId|rating|userId| prediction|
+-------+------+------+-----------+
|      3|   1.0|    28|-0.91167474|
|      0|   1.0|    22|  1.6049399|
|      2|   2.0|     1|  2.4619474|
|      6|   1.0|     1|-0.07001883|
|      3|   1.0|    13|   1.780483|
|      5|   1.0|     6|-0.85780734|
|      1|   1.0|     3|   2.019929|
|      6|   1.0|    20|  2.2937973|
|      0|   1.0|     5|  1.9613981|
|      4|   1.0|     5| 0.83987474|
|      2|   1.0|    15|  1.7851846|
|      5|   2.0|    15|  0.9745832|
|      2|   3.0|     9|   2.227317|
|      4|   1.0|     9|  1.3508558|
|      6|   1.0|    17|-0.43686342|
|      6|   1.0|     4|-0.27621064|
|      4|   1.0|     7|0.119006634|
|      4|   3.0|     2| -0.8730756|
|      6|   1.0|     2| 0.46432856|
|      5|   2.0|     0|  1.2426913|
+-------+------+------+-----------+
only showing top 20 rows



In [14]:
evalator = RegressionEvaluator(labelCol='rating', predictionCol='prediction', metricName='rmse')

In [15]:
rmse = evalator.evaluate(prediction)

In [16]:
print('RSME', rmse)

RSME 1.7150913768399716


In [24]:
singel_user = test.filter(test['userId'] == 11).select(['movieId', 'userId'])

In [25]:
singel_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|     18|    11|
|     21|    11|
|     23|    11|
|     35|    11|
|     36|    11|
|     59|    11|
|     72|    11|
|     78|    11|
|     80|    11|
|     82|    11|
|     88|    11|
|     97|    11|
+-------+------+



In [26]:
recommendation = model.transform(singel_user)

In [27]:
recommendation.orderBy('prediction', ascending=False).show()

+-------+------+-----------+
|movieId|userId| prediction|
+-------+------+-----------+
|     23|    11|   5.783869|
|     88|    11|  4.3596354|
|     18|    11|   4.085654|
|     36|    11|   3.143075|
|     82|    11|  2.8758168|
|     80|    11|  2.7181423|
|     21|    11|  2.7083695|
|     35|    11|    2.31091|
|     78|    11|  1.1010001|
|     97|    11|  0.6145925|
|     72|    11| 0.46659625|
|     59|    11|-0.27252108|
+-------+------+-----------+

